# Project 60: YOLOv5 Training

Disclaimer: modified from the duckietown dt_object_detection_trainign.ipynb

This notebook is for training of a YOLOv5 model for the duckiebot.


Utility functions for managing directories and running shell commands

In [ ]:
import os
import contextlib

# Contecxt manager: temporraily change the current working dir to 'name' dir
@contextlib.contextmanager
def directory(name):
  ret = os.getcwd()
  os.chdir(name)
  yield None
  os.chdir(ret)

import subprocess

# Run a shell command
def run(input, exception_on_failure=False):
  try:
    program_output = subprocess.check_output(f"{input}", shell=True, universal_newlines=True, stderr=subprocess.STDOUT)
  except Exception as e:
    if exception_on_failure:
      raise e
    program_output = e.output

    return program_output

# Run and print a shell command
def prun(input, exception_on_failure=False):
  x = run(input, exception_on_failure)
  print(x)
  return x

Create a temporary workspace

In [ ]:
import tempfile
SESSION_WORKSPACE = tempfile.mkdtemp()
print(f"Session workspace created at: {SESSION_WORKSPACE}")

Mount google drive

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')
DRIVE_PATH = "/content/drive/My Drive"

Prepare the dataset

In [ ]:
# Unzip the dataset
import shutil
import os


DATASET_DIR_NAME = "duckietown_object_detection_dataset"
DATASET_ZIP_NAME = f"{DATASET_DIR_NAME}.zip"
DATASET_DIR_PATH = os.path.join(SESSION_WORKSPACE, DATASET_DIR_NAME)
TRAIN_DIR = "train"
VALIDATION_DIR = "val"
IMAGES_DIR = "images"
LABELS_DIR = "labels"


def show_info(base_path: str):
  for l1 in [TRAIN_DIR, VALIDATION_DIR]:
    for l2 in [IMAGES_DIR, LABELS_DIR]:
      p = os.path.join(base_path, l1, l2)
      print(f"#Files in {l1}/{l2}: {len(os.listdir(p))}")


def unzip_dataset():
  # check zipped file
  zip_path = os.path.join(DRIVE_PATH, DATASET_ZIP_NAME)
  assert os.path.exists(zip_path), f"No zipped dataset found at {zip_path}! Abort!"

  # unzip the data
  print("Unpacking zipped data...")
  shutil.unpack_archive(zip_path, DATASET_DIR_PATH)
  print(f"Zipped dataset unpacked to {DATASET_DIR_PATH}")

  # show some info
  show_info(DATASET_DIR_PATH)


unzip_dataset()

Install torch and torchvision

In [ ]:
# change  working directory to the session workspace
os.chdir(SESSION_WORKSPACE)
print(f"PWD: {os.getcwd()}")

# install pytorch and torchvision (these versions must match those on the physical duckiebot)
!pip3 install torch==1.11.0 torchvision

Clone Yolov5

In [ ]:
!rm -rf ./yolov5
!git clone https://github.com/ultralytics/yolov5.git -b v7.0
!cd yolov5 && pip3 install -r requirements.txt

YOLOv5 training configuration file

In [ ]:
%%writefile ./yolov5/data/duckietown.yaml

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../duckietown_object_detection_dataset/train
val: ../duckietown_object_detection_dataset/val

# number of classes
nc: 5

# class names
names: [ 'duckie', 'cone', 'truck', 'bus','duckiebot' ]


# Training

In [ ]:
!cd yolov5 && python3 train.py --cfg ./models/yolov5n.yaml --img 416 --batch 16 --epochs 75 --data duckietown.yaml --weights yolov5n.pt -- hyp.scratch-low.yaml 

Get the model weights from the lastest run

In [ ]:
import numpy as np

all_exps = os.listdir("yolov5/runs/train")
all_exps_filtered = map(lambda x: int(x.replace("exp", "1")), filter(lambda x: x.startswith("exp"), all_exps))
all_exps_filtered = np.array(list(all_exps))
latest_exp_index = np.argmax(all_exps)
latest_exp = all_exps[latest_exp_index]
print(f"Latest exp is {latest_exp}")

prun(f"cp yolov5/runs/train/{latest_exp}/weights/best.pt yolov5/best.pt")
print(f"Marked the model from the latest run ({latest_exp}) as yolov5/best.pt.")

Upload model to Duckietown's cloud

In [ ]:
import sys
sys.path.insert(0, './yolov5')

model_name = "yolov5n"
model_local_path = "./yolov5/best.pt"
model_remote_path = f"courses/mooc/objdet/data/nn_models/{model_name}.pt"

DT_TOKEN = "dt1-3nT8KSoxVh4MnDRxovGLkXZDhPpgc4SzasJBTSxbRUfDguS-43dzqWFnWd8KBa1yev1g3UKnzVxZkkTbfeFCAD1kMCPQvvSVDYPfoXapvF29wVgdC7"

Install DCS client

In [ ]:
!pip3 install dt-data-api-daffy

Open a pointer to DT cloud and upload model

In [ ]:
import torch
from dt_data_api import DataClient, Storage

# open a pointer to our personal duckietown cloud space
client = DataClient(DT_TOKEN)
storage = client.storage("user")

# upload model
upload = storage.upload(model_local_path, model_remote_path)
upload.join()

Move the workspace to Google Drive

In [ ]:
import shutil

# Move the workspace directory to Google Drive
shutil.move(SESSION_WORKSPACE, '/content/drive/My Drive/')